In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [1]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.models import model_from_json
import os
import numpy
from keras import backend
backend.set_image_data_format('channels_first')
seed = 5
numpy.random.seed(seed)

Using TensorFlow backend.


In [3]:
'''
this part is required to solve
AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'experimental_list_devices'.
This error occurs because the experimental_list_devices is deprecated in tf 2.1.
'''
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback
print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)
def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus

tf.__version__ is 2.1.0
tf.keras.__version__ is: 2.2.4-tf


In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_test = x_test/255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [12]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(3,32,32), padding='same', activation='relu'))
model.add(Dropout(.2))
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(Dropout(.2))
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(Dropout(.2))
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.2))
model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.2))
model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(.2))
model.add(Dense(2048, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(.4))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(.4))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(.3))
model.add(Dense(num_classes, activation='softmax'))
epochs=5
lr=.01
decay=lr/epochs
sgd = SGD(lr=lr, momentum=.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_54 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 32, 16, 16)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 64, 16, 16)        18496     
_________________________________________________________________
dropout_55 (Dropout)         (None, 64, 16, 16)        0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 64, 16, 16)       

In [6]:
model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=epochs, batch_size=32, verbose=2)
scores = model.evaluate(x_test,y_test, verbose=0)
print('Accuracy: %.2f%%' % (scores[1]*100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
 - 14s - loss: 1.9724 - accuracy: 0.2622 - val_loss: 1.7277 - val_accuracy: 0.3670
Epoch 2/5
 - 11s - loss: 1.5948 - accuracy: 0.4123 - val_loss: 1.5774 - val_accuracy: 0.4315
Epoch 3/5
 - 10s - loss: 1.4514 - accuracy: 0.4706 - val_loss: 1.3948 - val_accuracy: 0.4959
Epoch 4/5
 - 10s - loss: 1.3728 - accuracy: 0.4989 - val_loss: 1.3332 - val_accuracy: 0.5155
Epoch 5/5
 - 10s - loss: 1.3194 - accuracy: 0.5220 - val_loss: 1.2952 - val_accuracy: 0.5352
Accuracy: 53.52%


In [7]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5')
print('Saved model to disk')

Saved model to disk


In [8]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('model.h5')
print("Loaded model from disk")

Loaded model from disk


In [9]:
print('Accuracy: %.2f%%' % (scores[1]*100))

Accuracy: 53.52%
